In [1]:
import pandas as pd
from entsoe import EntsoePandasClient
import os
from datetime import datetime
from requests_html import HTMLSession
import warnings
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost
import plotly.graph_objects as go
from sklearn.metrics import root_mean_squared_error
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import datetime
pd.set_option('display.max_columns', None)

# ENTSOE DATA

In [2]:
warnings.filterwarnings('ignore')

session = HTMLSession(verify=False)
client=EntsoePandasClient(api_key='70fec3b8-9274-4fa2-b777-93d8a0390cdb')

start = pd.Timestamp('20181201', tz='Europe/Paris')
end = pd.Timestamp('20240429', tz='Europe/Paris')
country_code = 'FR'

In [3]:
df_solar = client.query_generation(country_code=country_code, start=start, end=end, psr_type='B16').reset_index().rename(columns={"index":'Timestamp'})

In [4]:
solar_forecast = client.query_wind_and_solar_forecast(country_code=country_code, start=start, end=end, psr_type='B16').reset_index().rename(columns={"index":'Timestamp'})

In [5]:
def fillna_moving_average(df):
    if df.isna().any().sum()>0:
        return df.fillna((df.ffill() + df.bfill())/2, inplace=False)
    else:
        return df

In [6]:
solar_forecast['Timestamp'] = pd.to_datetime(solar_forecast['Timestamp'], utc=True).dt.tz_convert('Europe/Paris').dt.tz_localize(None)
solar_forecast.drop_duplicates(subset='Timestamp', keep='first',inplace=True)
solar_forecast = fillna_moving_average(solar_forecast.set_index('Timestamp').resample('h').mean()).reset_index()

In [7]:
df_solar['Timestamp'] = pd.to_datetime(df_solar['Timestamp'], utc=True).dt.tz_convert('Europe/Paris').dt.tz_localize(None)
df_solar.drop_duplicates(subset='Timestamp', keep='first',inplace=True)
df_solar = fillna_moving_average(df_solar.set_index('Timestamp').resample('h').mean()).reset_index()

In [ ]:
# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_solar['Timestamp'], y=df_solar['Solar'],
                    mode='lines',
                    name='True'))
fig.add_trace(go.Scatter(x=solar_forecast['Timestamp'], y=solar_forecast['Solar'],
                    mode='lines',
                    name='Forecast'))
fig.show()

In [ ]:
solar_forecast[solar_forecast['Timestamp'].between(pd.Timestamp('20230418'), pd.Timestamp('20230418 23:00:00'))]

In [ ]:
root_mean_squared_error(y_true=df_solar['Solar'], y_pred=solar_forecast['Solar'])

In [16]:
from functools import reduce

# METEO DATA

In [8]:
import requests_cache
from retry_requests import retry
import openmeteo_requests

In [9]:
def get_meteo_data(params):

    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    url = "https://archive-api.open-meteo.com/v1/archive"


    responses = openmeteo.weather_api(url, params=params) 

    response = responses[0]
    hourly = response.Hourly()

    hourly_data = {"Timestamp":pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit='s', utc=True).tz_convert(params['timezone']),
        end = pd.to_datetime(hourly.TimeEnd(), unit='s', utc=True).tz_convert(params['timezone']),
        freq = pd.Timedelta(seconds=hourly.Interval()),
        inclusive='right'
    )}
    
    for idx, variable in enumerate(params['hourly']):
        hourly_data[variable] = hourly.Variables(idx).ValuesAsNumpy()

    # print(hourly_data['date'])
    # print(hourly_data["apparent_temperature"])
    # print(responses[0])
    hourly_dataframe = pd.DataFrame(hourly_data)
    return hourly_dataframe


In [40]:
param_cestas = params = {
        "latitude": 44.7447,
        "longitude": -0.6819,
        "start_date": '2018-12-01',
        "end_date": '2024-04-28',
        "hourly": ["relative_humidity_2m", "apparent_temperature", "rain", "cloud_cover", "direct_radiation_instant", "global_tilted_irradiance_instant"],
        'timezone':'Europe/Paris'}
    

param_toul_rosieres = params = {
        "latitude": 48.7935,
        "longitude": 6.0011,
        "start_date": '2018-12-01',
        "end_date": '2024-04-28',
        "hourly": ["relative_humidity_2m", "apparent_temperature", "rain", "cloud_cover", "direct_radiation_instant", "global_tilted_irradiance_instant"],
        'timezone':'Europe/Paris'}

param_colle_mees = params = {
        "latitude": 44.0296,
        "longitude": 5.9764,
        "start_date": '2018-12-01',
        "end_date": '2024-04-28',
        "hourly": ["relative_humidity_2m", "apparent_temperature", "rain", "cloud_cover", "direct_radiation_instant", "global_tilted_irradiance_instant"],
        'timezone':'Europe/Paris'}

param_crucey_village = params = {
        "latitude": 48.6667,
        "longitude": 1.0833,
        "start_date": '2018-12-01',
        "end_date": '2024-04-28',
        "hourly": ["relative_humidity_2m", "apparent_temperature", "rain", "cloud_cover", "direct_radiation_instant", "global_tilted_irradiance_instant"],
        'timezone':'Europe/Paris'}

param_losse = params = {
        "latitude": 44.1086,
        "longitude": -0.1027,
        "start_date": '2018-12-01',
        "end_date": '2024-04-28',
        "hourly": ["relative_humidity_2m", "apparent_temperature", "rain", "cloud_cover", "direct_radiation_instant", "global_tilted_irradiance_instant"],
        'timezone':'Europe/Paris'}

In [41]:
solar_cestas = get_meteo_data(param_cestas)[:-1]
solar_toul_rosieres = get_meteo_data(param_toul_rosieres)[:-1]
solar_colle_mees = get_meteo_data(param_colle_mees)[:-1]
solar_crucey_village = get_meteo_data(param_crucey_village)[:-1]
solar_losse = get_meteo_data(param_losse)[:-1]

In [42]:
def processing_time_column(df, timestamp_column='Timestamp', timezone='Europe/Paris'):
    df[timestamp_column] = pd.to_datetime(df[timestamp_column], utc=True).dt.tz_convert(timezone).dt.tz_localize(None)
    df.drop_duplicates(subset=timestamp_column, keep='first',inplace=True)
    df = fillna_moving_average(df.set_index(timestamp_column).resample('h').mean()).reset_index()
    return df

In [43]:
solar_cestas = processing_time_column(solar_cestas)
solar_toul_rosieres = processing_time_column(solar_toul_rosieres)
solar_colle_mees = processing_time_column(solar_colle_mees)
solar_crucey_village = processing_time_column(solar_crucey_village)
solar_losse = processing_time_column(solar_losse)

In [44]:
list_dfs = [solar_cestas,solar_toul_rosieres,solar_colle_mees,solar_crucey_village,solar_losse]
list_city = ['cestas','toul','colle','crucey','losse']

In [45]:
solar_features = reduce(lambda df1,df2:pd.merge(df1,df2,on='Timestamp'),
                                           [f.set_index('Timestamp').add_suffix(f'_{city}').reset_index() for f,city in zip(list_dfs,list_city)])

In [46]:
solar_features

,Timestamp,relative_humidity_2m_cestas,apparent_temperature_cestas,rain_cestas,cloud_cover_cestas,direct_radiation_instant_cestas,global_tilted_irradiance_instant_cestas,relative_humidity_2m_toul,apparent_temperature_toul,rain_toul,...,rain_crucey,cloud_cover_crucey,direct_radiation_instant_crucey,global_tilted_irradiance_instant_crucey,relative_humidity_2m_losse,apparent_temperature_losse,rain_losse,cloud_cover_losse,direct_radiation_instant_losse,global_tilted_irradiance_instant_losse
0,2018-12-01 00:00:00,89.913597,5.454428,0.0,48.300003,0.000000,0.000000,96.902306,1.904276,0.0,...,0.0,32.700001,0.000000,0.000000,100.000000,4.616829,0.0,42.599998,0.000000,0.000000
1,2018-12-01 01:00:00,90.546890,5.587580,0.0,82.500000,0.000000,0.000000,96.892433,1.461233,0.0,...,0.0,18.000000,0.000000,0.000000,100.000000,4.661873,0.0,55.799999,0.000000,0.000000
2,2018-12-01 02:00:00,89.395363,6.784460,0.0,65.699997,0.000000,0.000000,96.225571,1.634677,0.0,...,0.0,33.299999,0.000000,0.000000,99.999992,4.600109,0.0,40.799999,0.000000,0.000000
3,2018-12-01 03:00:00,93.101585,6.362766,0.0,57.000000,0.000000,0.000000,97.233444,1.412157,0.0,...,0.0,70.200005,0.000000,0.000000,100.000000,4.476688,0.0,100.000000,0.000000,0.000000
4,2018-12-01 04:00:00,94.060593,6.274180,0.0,47.099998,0.000000,0.000000,97.570663,1.147175,0.0,...,0.0,94.199997,0.000000,0.000000,100.000000,4.411729,0.0,63.899998,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47419,2024-04-28 19:00:00,53.133617,12.730297,0.0,2.400000,364.440948,468.692078,72.180260,8.674788,0.0,...,0.0,55.500000,252.100616,360.643890,53.978024,13.406984,0.1,33.000000,240.148605,375.505096
47420,2024-04-28 20:00:00,54.088062,12.318329,0.0,0.000000,215.626083,298.000061,76.944420,8.524611,0.0,...,0.0,29.699999,133.371964,215.011765,61.390339,12.942366,0.2,70.199997,167.702454,240.721237
47421,2024-04-28 21:00:00,60.523956,11.714373,0.0,0.000000,92.219406,136.670486,81.424881,8.078767,0.0,...,0.0,36.299999,64.893158,107.040260,74.529396,13.346739,0.0,29.699999,53.249619,95.459679
47422,2024-04-28 22:00:00,71.581566,9.236286,0.0,0.000000,0.000000,0.000000,83.875954,7.459280,0.0,...,0.0,38.700001,0.738809,1.346588,75.625595,11.842661,0.0,27.599998,0.000000,0.000000


In [47]:
for mean_feature in ['humidity','temperature','cloud_cover']:
    drop_columns = solar_features.columns[solar_features.columns.str.contains(mean_feature)]
    solar_features[mean_feature] = solar_features[drop_columns].mean(axis=1)
    solar_features.drop(columns=drop_columns, inplace=True)

for sum_feature in ['direct_radiation','tilted_irradiance','rain']:
    drop_columns = solar_features.columns[solar_features.columns.str.contains(sum_feature)]
    solar_features[sum_feature] = solar_features[drop_columns].sum(axis=1)
    solar_features.drop(columns=drop_columns, inplace=True)

# Training classique

In [ ]:
list_dicts = [{'feature':['Solar'],
               'dataset':df_solar,
               'hours':None,
               'days':[0,2,3,4,5,6,7]},
               {"feature":['humidity'],
                'dataset':solar_features,
                'hours':[0],
                'days':[2,3,4,5,6,7]},
                {"feature":['temperature'],
                 'dataset':solar_features,
                 'hours':[0],
                 'days':[2,3,4,5,6,7]},
                 {"feature":['cloud_cover'],
                 'dataset':solar_features,
                 'hours':[0],
                 'days':[2,3,4,5,6,7]},
                {"feature":['direct_radiation'],
                 'dataset':solar_features,
                 'hours':[0],
                 'days':[2,3,4,5,6,7]},
                 {"feature":['tilted_irradiance'],
                 'dataset':solar_features,
                 'hours':[0],
                 'days':[2,3,4,5,6,7]},
                 {"feature":['rain'],
                 'dataset':solar_features,
                 'hours':[0],
                 'days':[2,3,4,5,6,7],
                 }
                ]


data = LagFeatures(list_dicts)
dataset = data.create_dataset()


In [ ]:
dataset

In [24]:
def get_season(x):
    if x in [12,1,2]:
        return 1
    if x in [3,4,5]:
        return 2
    if x in [6,7,8]:
        return 3
    if x in [9,10,11]:
        return 4
    
def is_weekend(x):
    if x < 5 :
        return 0
    else :
        return 1

In [ ]:
dataset['season'] = dataset['Timestamp'].dt.month.apply(lambda x:get_season(x))
dataset['hour'] = dataset['Timestamp'].dt.hour
dataset['is_weekend'] = dataset['Timestamp'].dt.weekday.apply(lambda x:is_weekend(x))


In [ ]:
Target_value = ['Solar_J-0']
Features = dataset.columns[~dataset.columns.str.contains(f'{Target_value[0]}|Timestamp')]

In [ ]:
list(Features[~Features.str.contains('season|hour|is_weekend')])

In [ ]:
numerical_features = list(Features[~Features.str.contains('season|hour|is_weekend')])
categorical_features = ['season', 'hour', 'is_weekend']
numerical_pipeline = make_pipeline(MinMaxScaler())
categorical_pipeline = make_pipeline(OneHotEncoder())

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                   (categorical_pipeline, categorical_features))

In [ ]:
from sklearn.ensemble import StackingRegressor

In [ ]:

train = dataset[dataset['Timestamp']<= pd.Timestamp('20221230 23:00:00')]
test = dataset[dataset['Timestamp']> pd.Timestamp('20221231 23:00:00')]
X_train, y_train = train[Features], train[Target_value]
X_test, y_test = test[Features], test[Target_value]


# estimators = [('xgboost', xgboost.XGBRegressor(random_state=42, n_estimators=500, max_depth=None))]
# model = make_pipeline(preprocessor, StackingRegressor(estimators=estimators,
#                                                       final_estimator = xgboost.XGBRegressor(random_state=42, n_estimators=500, max_depth=None)))
model = make_pipeline(preprocessor,xgboost.XGBRegressor(random_state=42, n_estimators=500, max_depth=None))

#model = LinearRegression()
#model = make_pipeline(preprocessor,RandomForestRegressor(random_state=42, n_estimators=250, max_depth=None))
model = model.fit(X_train, y_train)

#y_pred = pd.DataFrame(scaler_Y.inverse_transform(model.predict(X_test).reshape((-1,1))))[0]
y_pred = pd.DataFrame(model.predict(X_test).reshape((-1,1)))[0]
fig = go.Figure()
fig.add_trace(go.Scatter(x=test['Timestamp'], y=y_test['Solar_J-0'],
                    mode='lines',
                    name='true'))
fig.add_trace(go.Scatter(x=test['Timestamp'], y=y_pred,
                    mode='lines',
                    name='predictions'))

rmse = root_mean_squared_error(y_true=y_test, y_pred=y_pred)

fig.update_layout(title=f'RMSE : {rmse}')
fig.show()

In [ ]:
# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_solar[df_solar['Timestamp']>=pd.Timestamp('20230101')]['Timestamp'], y=df_solar[df_solar['Timestamp']>=pd.Timestamp('20230101')]['Solar'],
                    mode='lines',
                    name='True'))

fig.add_trace(go.Scatter(x=solar_forecast[solar_forecast['Timestamp']>=pd.Timestamp('20230101')]['Timestamp'], y=solar_forecast[solar_forecast['Timestamp']>=pd.Timestamp('20230101')]['Solar'],
                    mode='lines',
                    name='Forecast'))

rmse = root_mean_squared_error(y_true=df_solar[(df_solar['Timestamp']>=pd.Timestamp('20230101')) & (df_solar['Timestamp'].dt.date != datetime.date(year=2023, month=4, day=18))]['Solar'], y_pred=solar_forecast[(solar_forecast['Timestamp']>=pd.Timestamp('20230101'))& (solar_forecast['Timestamp'].dt.date != datetime.date(year=2023, month=4, day=18))]['Solar'])

fig.update_layout(title=f'RMSE : {rmse}')

fig.show()

In [ ]:
root_mean_squared_error(y_true=solar_forecast[solar_forecast['Timestamp']>=pd.Timestamp('20230101')]['Solar'], y_pred=y_pred)

# Rolling training set

In [ ]:
forecast = pd.DataFrame(index=test['Timestamp'], columns=['Solar_J-0'])

In [ ]:
# dataset.set_index('Timestamp', inplace=True)

In [ ]:
dataset['hour'] = dataset['Timestamp'].dt.hour

In [ ]:
dataset

In [ ]:
# Training
models = {}
Target_value = ['Solar_J-0']
Features = dataset.columns[~dataset.columns.str.contains(f'{Target_value[0]}|Timestamp|hour')]
numerical_features = list(Features[~Features.str.contains('season|hour|is_weekend')])
categorical_features = ['season', 'is_weekend']
numerical_pipeline = make_pipeline(StandardScaler())
categorical_pipeline = make_pipeline(OneHotEncoder())

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                   (categorical_pipeline, categorical_features))
for hour in range(24):
    train = dataset[dataset['Timestamp']<= pd.Timestamp('20221230 23:00:00')].query(f"hour=={hour}")
    X_train, y_train = train[Features], train[Target_value]


    #model = make_pipeline(preprocessor, xgboost.XGBRegressor(random_state=42, n_estimators=500, max_depth=None))
    model = make_pipeline(preprocessor,LinearRegression())
    model = model.fit(X_train, y_train)
    models[hour] = model


    

In [ ]:
for date in tqdm(sorted(list(set(forecast.index.date)))):
    predictions = []
    for hour in range(24): 
        X_test = test[test['Timestamp'].dt.date==date].query(f'hour=={hour}')[Features]
        y_pred = models[hour].predict(X_test)
        predictions.append(y_pred[0][0])
  
    forecast.loc[pd.Timestamp(date):pd.Timestamp(date)+pd.Timedelta(hours=23),'Solar_J-0'] = predictions
                                         

In [ ]:
forecast

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test['Timestamp'], y=test['Solar_J-0'],
                    mode='lines',
                    name='true'))
fig.add_trace(go.Scatter(x=test['Timestamp'], y=forecast['Solar_J-0'],
                    mode='lines',
                    name='predictions'))

rmse = root_mean_squared_error(y_true=test['Solar_J-0'], y_pred=forecast['Solar_J-0'])

fig.update_layout(title=f'RMSE : {rmse}')
fig.show()